# Машинное обучение, ШАД
## Домашнее задание 9: Кластеризация

In [32]:
!pip install catboost
!pip install numpy==1.26.4

ERROR: Could not find a version that satisfies the requirement catboost (from versions: none)
ERROR: No matching distribution found for catboost
ERROR: Could not find a version that satisfies the requirement numpy==1.26.4 (from versions: none)
ERROR: No matching distribution found for numpy==1.26.4


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from typing import Union
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor, Pool
# допиши необходимые импорты

sns.set_theme(palette="Set2", font_scale=1.2)

ModuleNotFoundError: No module named 'catboost'

---
### Задача 1.

Вспомним задачу 2 из задания 5 🤩

Загрузите данные и разделите на две части.

In [7]:
Coordinate = Union[float, np.ndarray, pd.Series]

def haversine(lat1: Coordinate, lon1: Coordinate, lat2: Coordinate, lon2: Coordinate) -> np.ndarray:
    """
    Вычисляет расстояние между двумя точками на Земле по формуле гаверсина.

    Параметры
    ----------
    lat1: Широта первой точки в градусах.
    lon1: Долгота первой точки в градусах.
    lat2: Широта второй точки в градусах.
    lon2: Долгота второй точки в градусах.

    Возвращает
    -------
    np.ndarray
        Расстояние между точками в километрах.
    """
    R = 6371.0  # Средний радиус Земли в километрах

    # Преобразование градусов в радианы для вычислений
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)

    # Основная формула Гаверсина
    a = np.sin(dphi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda / 2) ** 2
    # Использование arctan2 для большей точности
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    return R * c

In [11]:
dataset = pd.read_csv('./datasets/train.csv', parse_dates=["pickup_datetime", "dropoff_datetime"])
dataset.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


Чуть-чуть преобразуем признаки и таргет, как мы это делали в ДЗ 5. А так же почистим обучающие данные, как и в ДЗ 5.

In [13]:
dataset["trip_duration_min"] = dataset["trip_duration"] / 60.0

# Расстояние
dataset["distance_km"] = haversine(
    dataset["pickup_latitude"],
    dataset["pickup_longitude"],
    dataset["dropoff_latitude"],
    dataset["dropoff_longitude"],
)

# День недели (0 = понедельник)
dataset["weekday"] = dataset["pickup_datetime"].dt.weekday
# Выходные
dataset["is_weekend"] = (dataset["weekday"] >= 5).astype(int)
# Часы
dataset["hour"] = dataset["pickup_datetime"].dt.hour
# Утренний час-пик
dataset["is_peak_morning"] = dataset["hour"].between(7, 10).astype(int)
# Вечерний час-пик
dataset["is_peak_evening"] = dataset["hour"].between(16, 20).astype(int)

# некоторые признаки можно убрать
dataset["dist_peak_morning"] = dataset["distance_km"] * dataset["is_peak_morning"]
dataset["dist_peak_evening"] = dataset["distance_km"] * dataset["is_peak_evening"]
dataset["dist_weekend"] = dataset["distance_km"] * dataset["is_weekend"]
dataset["dist2"] = dataset["distance_km"] ** 2
dataset["log_dist"] = np.log1p(dataset["distance_km"])

# придумайте свои признаки исходя из постановки задачи
uws_lat_min, uws_lat_max = 40.773, 40.805
uws_lon_min, uws_lon_max = -73.99, -73.96

dataset["pickup_uws"] = ((dataset["pickup_latitude"] >= uws_lat_min) & (dataset["pickup_latitude"] <= uws_lat_max) &
                    (dataset["pickup_longitude"] >= uws_lon_min) & (dataset["pickup_longitude"] <= uws_lon_max)).astype(int)

dataset["dropoff_uws"] = ((dataset["dropoff_latitude"] >= uws_lat_min) & (dataset["dropoff_latitude"] <= uws_lat_max) &
                     (dataset["dropoff_longitude"] >= uws_lon_min) & (dataset["dropoff_longitude"] <= uws_lon_max)).astype(int)

In [16]:
x_data = dataset.drop(columns=["id", "trip_duration", "pickup_datetime", "dropoff_datetime", "trip_duration_min", "store_and_fwd_flag", "pickup_latitude", "pickup_longitude", "dropoff_latitude", "dropoff_longitude"])
y_data = dataset["trip_duration_min"]

x_train, x_test, y_train, y_test = train_test_split( x_data, y_data, test_size=0.2, random_state=42)

In [17]:
temp_df = pd.concat([x_train, y_train], axis=1)


clean_temp_df = temp_df[(temp_df["trip_duration_min"] < 70)]
clean_temp_df = clean_temp_df[(clean_temp_df["distance_km"] < 30)]

clean_x_train = clean_temp_df.drop(columns=["trip_duration_min"])
clean_y_train = clean_temp_df["trip_duration_min"]

print(f"Размер обучающей выборки после удаления выбросов: {clean_x_train.shape[0]}")

prost_deleted = (temp_df.shape[0] - clean_y_train.shape[0]) / temp_df.shape[0] * 100
print(f"Процент данных, которое пришлось удалить: {prost_deleted:.2}%")

Размер обучающей выборки после удаления выбросов: 1161038
Процент данных, которое пришлось удалить: 0.5%


In [18]:
x_train = clean_x_train
y_train = clean_y_train

In [19]:
temp_df = pd.concat([x_train, y_train], axis=1)

clean_temp_df = temp_df[(temp_df["trip_duration_min"] < 60)]
clean_temp_df = clean_temp_df[(clean_temp_df["distance_km"] < 15)]

clean_x_train = clean_temp_df.drop(columns=["trip_duration_min"])
clean_y_train = clean_temp_df["trip_duration_min"]

print(f"Размер обучающей выборки после удаления выбросов: {clean_x_train.shape[0]}")

prost_deleted = (temp_df.shape[0] - clean_y_train.shape[0]) / temp_df.shape[0] * 100
print(f"Процент данных, которое пришлось удалить: {prost_deleted:.2}%")

Размер обучающей выборки после удаления выбросов: 1126882
Процент данных, которое пришлось удалить: 2.9%


In [20]:
x_train = clean_x_train
y_train = clean_y_train

In [21]:
temp_df = pd.concat([x_train, y_train], axis=1)

clean_temp_df = temp_df[(temp_df["distance_km"] > 0.5)]

clean_x_train = clean_temp_df.drop(columns=["trip_duration_min"])
clean_y_train = clean_temp_df["trip_duration_min"]

print(f"Размер обучающей выборки после удаления выбросов: {clean_x_train.shape[0]}")

prost_deleted = (temp_df.shape[0] - clean_y_train.shape[0]) / temp_df.shape[0] * 100
print(f"Процент данных, которое пришлось удалить: {prost_deleted:.2}%")

Размер обучающей выборки после удаления выбросов: 1087957
Процент данных, которое пришлось удалить: 3.5%


In [22]:
x_train = clean_x_train
y_train = clean_y_train

In [23]:
temp_df = pd.concat([x_train, y_train], axis=1)

clean_temp_df = temp_df[(temp_df["distance_km"] / (temp_df["trip_duration_min"] / 60) < 50)]

clean_x_train = clean_temp_df.drop(columns=["trip_duration_min"])
clean_y_train = clean_temp_df["trip_duration_min"]

print(f"Размер обучающей выборки после удаления выбросов: {clean_x_train.shape[0]}")

prost_deleted = (temp_df.shape[0] - clean_y_train.shape[0]) / temp_df.shape[0] * 100
print(f"Процент данных, которое пришлось удалить: {prost_deleted:.2}%")

Размер обучающей выборки после удаления выбросов: 1087028
Процент данных, которое пришлось удалить: 0.085%


In [24]:
x_train = clean_x_train
y_train = clean_y_train

Вспомним, что в ДЗ 5 говорили про чистку также тестовых данных. Поэтому для чистоты сравнения сделаем так же. 

In [25]:
temp_df = pd.concat([x_test, y_test], axis=1)

clean_temp_df = temp_df[(temp_df["trip_duration_min"] < 60)]
clean_temp_df = clean_temp_df[(clean_temp_df["distance_km"] < 15)]

clean_x_test = clean_temp_df.drop(columns=["trip_duration_min"])
clean_y_test = clean_temp_df["trip_duration_min"]

x_test = clean_x_test
y_test = clean_y_test

In [26]:
temp_df = pd.concat([x_test, y_test], axis=1)

clean_temp_df = temp_df[(temp_df["distance_km"] > 0.5)]

clean_x_test = clean_temp_df.drop(columns=["trip_duration_min"])
clean_y_test = clean_temp_df["trip_duration_min"]

x_test = clean_x_test
y_test = clean_y_test

In [27]:
temp_df = pd.concat([x_test, y_test], axis=1)

clean_temp_df = temp_df[(temp_df["distance_km"] / (temp_df["trip_duration_min"] / 60) < 50)]

clean_x_test = clean_temp_df.drop(columns=["trip_duration_min"])
clean_y_test = clean_temp_df["trip_duration_min"]

x_test = clean_x_test
y_test = clean_y_test

Постройте предсказание длительности поездки с помощью CatBoost. Не забудьте, что CatBoost умеет эффективно работать с категориальными признаками. Предсказание должно быть монотонным по длине поездки. Иначе говоря, при увеличении расстояния поездки и *фиксированных* остальных признаках *среднее* время поездки не может уменьшаться.

Оцените качество полученного предсказания по MAPE на тестовой выборке и сравните с полученным значением из ДЗ 5.

In [ ]:
rng = np.random 

Обучите еще две модели CatBoost для построения предсказательного интервала для фактического времени поездки. Постройте интервал по сетке значений расстояния аналогично ДЗ 5.

In [ ]:
...

Оцените долю покрытия предсказательным интервалом *фактического* времени поездки по *всем* точкам тестовой выборки.

In [ ]:
...

**Выводы:**

...

---
### Задача 2.

Докажите, что метод KMeans для евклидовой метрики делит все пространство объектов на выпуклые многоугольники, возможно, неограниченные.

Если вы используете какие-то теоремы и свойства, нужно их четко сформулировать, определить все обозначения, а также привести четкое обоснование, как вы пользуетесь соответствующими свойствами.

...

---
### Задача 3.



В этой задаче мы попробуем кластеризовать различными способами изображения котиков 😻 из датасета с занятия.

Скачайте данные, загрузите их и отрисуйте несколько примеров. Если вы используете Colab, оптимальнее всего будет закинуть zip-архив `cats.zip` в файловое пространство (кнопка папки на левой панели) и распаковать его с помощью `!unzip cats.zip`.

In [ ]:
...

#### 1. Свойства метрики в пространстве котиков

Как мы знаем из лекции, в пространствах большой размерности расстояния между случайными объектами становятся неинформативными. Эта проблема известна как *проклятие размерности*, и она влечет соответствующие ограничения на использование методов, основанных на использовании расстоянии между объектами.

Прежде всего давайте исследуем, наблюдается ли проблема проклятия размерности в пространстве котиков. Для этого выберите не менее 10000 случайных пар изображений и посчитайте расстояния между ними. Нормируйте все расстояния, поделив на среднее полученных расстояний. Визуализируйте гистограмму или <a href="https://thetahat.ru/courses/python/07" target="_top">KDE-оценку плотности</a> нормированных расстояний.

In [ ]:
...

Повторите те же действия к результату применения PCA, рассмотрев разное количество компонент, например, для 30, 100 и 500. Обратите внимание, что PCA нужно обучать на всех изображениях, а не только для тех, для которых вы будете считать расстояния. Примеры обучения PCA можно посмотреть в ноутбуке с занятия.

In [ ]:
...

Сравните результаты и сделайте выводы

In [ ]:
...

#### 2. Кластеризация котиков 🙀

На результатах применения PCA обучите метод KMeans из sklearn. Не стоит брать больше 100 главных компонент. Число кластеров подберите по метрике ситуэт (`silhouette_score` в sklearn), не берите больше 5-8 кластеров.

In [ ]:
...

Нарисуйте по 10-20 случайных изображений для каждого из этих кластеров. Визуализируйте по 1-2 строчки изображений для каждого кластера.

In [ ]:
...

Посмотрите на полученные кластеры и опишите отличительные черты для каждого кластера

...

**Общие выводы по задаче**

...

---
© 2025 команда <a href="https://thetahat.ru/">ThetaHat</a> для курса ML-1 ШАД